Creating Tables

In [81]:
CREATE TABLE Deaths (
    iso_code varchar(20),
    continent varchar(30),
    location varchar(155),
    date date,
    population BIGINT,
    total_cases int,
    new_cases int,
    new_cases_smoothed float,
    total_deaths int,
    new_deaths int,
    new_deaths_smoothed float,
    total_cases_per_million float,
    new_cases_per_million float,
    new_cases_smoothed_per_million float,
    total_deaths_per_million float,
    new_deaths_per_million float,
    new_deaths_smoothed_per_million float,
    reproduction_rate float,
    icu_patients int,
    icu_patients_per_million float,
    hosp_patients int,
    hosp_patients_per_million float,
    weekly_icu_admissions float,
    weekly_icu_admissions_per_million float
);

CREATE TABLE Vaccinations (
    iso_code varchar(20),
    continent varchar(30),
    location varchar(155),
    date date,
    new_tests int,
    total_tests int,
    total_tests_per_thousand float,
    new_tests_per_thousand float,
    new_tests_smoothed int,
    new_tests_smoothed_per_thousand float,
    positive_rate float,
    tests_per_case float,
    tests_units varchar(16),
    total_vaccinations BIGINT,
    people_vaccinated int,
    people_fully_vaccinated int,
    new_vaccinations INT,
    new_vaccinations_smoothed float,
    total_vaccinations_per_hundred float,
    people_vaccinated_per_hundred float,
    people_fully_vaccinated_per_hundred float,
    new_vaccinations_smoothed_per_million float,
    stringency_index float,
    population_density float,
    median_age float,
    aged_65_older float,
    aged_70_older float,
    gdp_per_capita float,
    extreme_poverty float,
    cardiovasc_death_rate float,
    diabetes_prevalence float,
    female_smokers float,
    male_smokers float,
    handwashing_facilitites float,
    hospital_beds_per_thousand float,
    life_expectancy float,
    human_development_index float
);

: Msg 2714, Level 16, State 6, Line 1
There is already an object named 'Deaths' in the database.

Total execution time: 00:00:00.257

Create Master Key

In [24]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD='ThisIsMyPassword123!' ;

: Msg 15578, Level 16, State 1, Line 1
There is already a master key in the database. Please drop it before performing this statement.

Total execution time: 00:00:00.052

Create Databse Scoped Credential

In [30]:
CREATE DATABASE SCOPED CREDENTIAL blobcred
WITH IDENTITY = 'SHARED ACCESS SIGNATURE',
SECRET = 'sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2023-06-22T08:15:44Z&st=2023-06-22T00:15:44Z&spr=https,http&sig=zOhaubFIcyH8Kr%2BGjZEptG%2Bpd%2BCkJbXb4hBar3Q%2BbAw%3D'

: Msg 15530, Level 16, State 1, Line 1
The credential with name "blobcred" already exists.

Total execution time: 00:00:00.065

Create Data Source From blob

In [82]:
CREATE EXTERNAL DATA SOURCE covidData_blob
WITH (TYPE = BLOB_STORAGE, 
LOCATION = 'https://personal98.blob.core.windows.net/personal-storage',
CREDENTIAL = blobcred
);

: Msg 46502, Level 16, State 1, Line 1
Type with name 'covidData_blob' already exists.

Total execution time: 00:00:00.082

Loading Data

In [83]:
SET NOCOUNT ON
BULK INSERT Deaths FROM 'CovidDeathsTrimmed.csv'
WITH (
    DATA_SOURCE = 'covidData_blob',
    DATAFILETYPE = 'char',
    FIRSTROW = 2, -- Skip the header row if necessary
    FIELDTERMINATOR = ',', -- Specify the CSV field delimiter
    ROWTERMINATOR = '\n', -- Specify the row delimiter
    BATCHSIZE=10000, -- reduce network traffic
    TABLOCK -- minimize log records
);



Commands completed successfully.

Total execution time: 00:00:15.597

In [84]:
BULK INSERT Vaccinations FROM 'CovidVaccinations.csv'
WITH (
    DATA_SOURCE = 'covidData_blob',
    DATAFILETYPE = 'char',
    CODEPAGE = 65001,
    FIRSTROW = 2, 
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '\n',
    BATCHSIZE=10000,
    TABLOCK 
);

Commands completed successfully.

Total execution time: 00:00:25.228